#  Gradio 챗봇 구현 (간단한 QA 애플리케이션)

### **학습 목표**
1. Gradio ChatInterface의 다양한 기능을 이해한다
2. LangChain LCEL을 활용한 챗봇 체인 구성 방법을 학습한다
3. 채팅 히스토리를 관리하고 컨텍스트를 유지하는 방법을 익힌다
4. 멀티모달 입력(텍스트+이미지)을 처리하는 챗봇을 구현한다

---

##  환경 설정

- 필수 라이브러리
    - gradio
    - langchain 
    - langchain-openai 
    - langchain-google-genai 
    - python-dotenv

- `.env` 파일에 다음 API 키 설정

    ```
    OPENAI_API_KEY=your_openai_key
    GOOGLE_API_KEY=your_google_key
    ```

In [1]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

True

## Simple QA Chain  

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 파이썬(Python) 코드 작성을 도와주는 AI 어시스턴트입니다."),
    ("human", "{user_input}")
])

# LLM 모델 정의
model = ChatOpenAI(
    model="gpt-4.1-nano", 
    temperature=0.3, 
    )

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()

# 체인 실행
response = chain.invoke({
    "user_input": "파이썬에서 리스트를 정렬하는 방법은 무엇인가요?"
})

# AI의 응답 텍스트를 출력 
print(response)

파이썬에서 리스트를 정렬하는 방법에는 여러 가지가 있습니다. 가장 일반적인 방법은 `sort()` 메서드와 `sorted()` 함수를 사용하는 것입니다.

1. **`list.sort()` 메서드**  
- 리스트 자체를 정렬하며, 정렬된 리스트를 반환하지 않고 원본 리스트를 변경합니다.  
- 사용법:  
```python
my_list = [3, 1, 4, 2]
my_list.sort()
print(my_list)  # 출력: [1, 2, 3, 4]
```

2. **`sorted()` 함수**  
- 원본 리스트는 변경하지 않고 정렬된 새 리스트를 반환합니다.  
- 사용법:  
```python
my_list = [3, 1, 4, 2]
sorted_list = sorted(my_list)
print(sorted_list)  # 출력: [1, 2, 3, 4]
```

### 정렬 기준 지정하기
- **`key` 인자**: 정렬 기준을 지정할 수 있습니다.  
- **`reverse` 인자**: 내림차순 정렬을 원할 때 사용합니다 (`True`).

예제: 문자열 길이 기준으로 정렬  
```python
words = ['apple', 'banana', 'cherry']
words_sorted = sorted(words, key=len)
print(words_sorted)  # 출력: ['apple', 'cherry', 'banana']
```

내림차순 정렬 예제:  
```python
numbers = [1, 3, 2, 4]
numbers_sorted_desc = sorted(numbers, reverse=True)
print(numbers_sorted_desc)  # 출력: [4, 3, 2, 1]
```

이 방법들을 활용해서 리스트를 원하는 방식으로 정렬하실 수 있습니다!


In [4]:
# 마크다운 출력
from IPython.display import display, Markdown

display(Markdown(response))

파이썬에서 리스트를 정렬하는 방법에는 여러 가지가 있습니다. 가장 일반적인 방법은 `sort()` 메서드와 `sorted()` 함수를 사용하는 것입니다.

1. **`list.sort()` 메서드**  
- 리스트 자체를 정렬하며, 정렬된 리스트를 반환하지 않고 원본 리스트를 변경합니다.  
- 사용법:  
```python
my_list = [3, 1, 4, 2]
my_list.sort()
print(my_list)  # 출력: [1, 2, 3, 4]
```

2. **`sorted()` 함수**  
- 원본 리스트는 변경하지 않고 정렬된 새 리스트를 반환합니다.  
- 사용법:  
```python
my_list = [3, 1, 4, 2]
sorted_list = sorted(my_list)
print(sorted_list)  # 출력: [1, 2, 3, 4]
```

### 정렬 기준 지정하기
- **`key` 인자**: 정렬 기준을 지정할 수 있습니다.  
- **`reverse` 인자**: 내림차순 정렬을 원할 때 사용합니다 (`True`).

예제: 문자열 길이 기준으로 정렬  
```python
words = ['apple', 'banana', 'cherry']
words_sorted = sorted(words, key=len)
print(words_sorted)  # 출력: ['apple', 'cherry', 'banana']
```

내림차순 정렬 예제:  
```python
numbers = [1, 3, 2, 4]
numbers_sorted_desc = sorted(numbers, reverse=True)
print(numbers_sorted_desc)  # 출력: [4, 3, 2, 1]
```

이 방법들을 활용해서 리스트를 원하는 방식으로 정렬하실 수 있습니다!

## Gradio ChatInterface  
- 설치: pip install gradio --upgrade , uv add gradio

### 1) 기본 구조

In [7]:
import gradio as gr

# 챗봇 함수 정의
def chat_function(message, history):
    # 메시지 출력
    print(f"입력 메시지: {message}")
    print("-"*40)
    # 채팅 히스토리 출력
    print(f"채팅 히스토리:")
    for chat in history:
        print(f"사용자: {chat['role']}, 메시지: {chat['content']}")
        print("-"*40)
    return "응답 메시지"

# 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=chat_function,  # 실행할 함수
    analytics_enabled=False,  # 사용 정보 제공 여부
)

# 챗봇 인터페이스 실행
demo.launch()

"""
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
"""

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


'\n* Running on local URL:  http://127.0.0.1:7860\n* To create a public link, set `share=True` in `launch()`.\n'

입력 메시지: hello
----------------------------------------
채팅 히스토리:
입력 메시지: who are you?
----------------------------------------
채팅 히스토리:
사용자: user, 메시지: [{'text': 'hello', 'type': 'text'}]
----------------------------------------
사용자: assistant, 메시지: [{'text': '응답 메시지', 'type': 'text'}]
----------------------------------------
입력 메시지: ok
----------------------------------------
채팅 히스토리:
사용자: user, 메시지: [{'text': 'hello', 'type': 'text'}]
----------------------------------------
사용자: assistant, 메시지: [{'text': '응답 메시지', 'type': 'text'}]
----------------------------------------
사용자: user, 메시지: [{'text': 'who are you?', 'type': 'text'}]
----------------------------------------
사용자: assistant, 메시지: [{'text': '응답 메시지', 'type': 'text'}]
----------------------------------------


In [8]:
# 인터페이스 종료
demo.close()

Closing server running on port: 7860


### 2) 간단한 예제: Echo 챗봇

In [10]:
def echo_bot(message, history):
    return f"당신이 입력한 메시지: {message}"

demo = gr.ChatInterface(
    fn=echo_bot,
    title="Echo 챗봇",
    description="입력한 메시지를 그대로 되돌려주는 챗봇입니다.",
    analytics_enabled=False,  
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [11]:
demo.close()

Closing server running on port: 7861


### 3) 스트리밍 응답

In [13]:
# 스트리밍 챗봇 함수 정의
import time

def streaming_bot(message, history):
    response = f"처리 중인 메시지: {message}"
    for i in range(len(response)):
        time.sleep(0.1)          # 0.1초 대기
        yield response[:i+1]

In [14]:
# 스트리밍 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=streaming_bot,
    title="스트리밍 챗봇",
    description="입력한 메시지를 한 글자씩 처리하는 챗봇입니다.",
    analytics_enabled=False,  
)

# 스트리밍 챗봇 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [15]:
demo.close()

Closing server running on port: 7861


### 4) 추가 입력 컴포넌트
- 최대 응답 길이 등 기타 설정을 위한 추가 입력

In [24]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 파이썬(Python) 코드 작성을 도와주는 AI 어시스턴트입니다."),
    ("human", "{user_input}")
])


# 챗봇 함수 정의
def chat_function(message, history, model, temperature):

    if model == "gpt-4.1-nano":
        model = ChatOpenAI(model=model, temperature=temperature)
    elif model == "gemini-2.5-flash-lite":
        model = ChatGoogleGenerativeAI(model=model, temperature=temperature)

    chain = prompt | model | StrOutputParser()

    response = chain.invoke({
        "user_input": message
    })
    return response

# 챗봇 인터페이스 생성
with gr.Blocks() as demo:
    model_selector = gr.Dropdown(["gpt-4.1-nano", "gemini-2.5-flash-lite"], label="모델 선택")
    slider = gr.Slider(0.0, 1.0, label="Temperature", value=0.3, step=0.1, render=False)   

    gr.ChatInterface(
        fn=chat_function, 
        additional_inputs=[model_selector, slider],
        analytics_enabled=False,  
    )

# 챗봇 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [25]:
demo.close()

Closing server running on port: 7862


### 5) 예시 질문 설정

In [19]:
# 스트리밍 챗봇 인터페이스 생성
demo = gr.ChatInterface(
    fn=streaming_bot,
    title="스트리밍 챗봇",
    description="입력한 메시지를 한 글자씩 처리하는 챗봇입니다.",
    analytics_enabled=False,  
    examples=[
        "파이썬 코드를 작성하는 방법을 알려주세요",
        "파이썬에서 리스트를 정렬하는 방법은 무엇인가요?",
    ]    
)

# 스트리밍 챗봇 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [20]:
demo.close()    

Closing server running on port: 7862


### 6) 멀티모달 기능
- `multimodal=True` 옵션
- 이미지나 파일을 처리할 수 있는 멀티모달 챗봇 구현

- message 파라미터:
    ```python
    {
        "text": "user input", 
        "files": [
            "updated_file_1_path.ext",
            "updated_file_2_path.ext", 
            ...
        ]
    }
    ```
- history 파라미터:
    ```python
    [
        {"role": "user", "content": ("cat1.png")},
        {"role": "user", "content": ("cat2.png")},
        {"role": "user", "content": "What's the difference between these two images?"},
    ]
    ```

In [ ]:
import gradio as gr
import base64
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

def convert_to_url(image_path):
    """이미지를 URL 형식으로 변환"""
    with open(image_path, "rb") as image_file:
        # 이미지를 base64로 인코딩
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return f"data:image/jpeg;base64,{encoded_string}"

def multimodal_bot(message, history):

    model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")
    
    if isinstance(message, dict):
        # 텍스트와 파일 추출
        text = message.get("text", "")
        
        # 히스토리와 현재 메시지에서 모든 파일 경로 추출
        filepath_list = []
        
        # 히스토리에서 이미지 파일 추출
        print("History:", history)  # 디버깅용
        for exchange in history:
            # user_message = exchange[0]
            # if isinstance(user_message, tuple):  # 이미지 메시지 확인
            #     filepath_list.append(user_message[0])
            # 새로운 Gradio 형식: exchange는 딕셔너리
            if isinstance(exchange, dict) and 'content' in exchange:
                content = exchange['content']
                # content는 리스트 형태
                if isinstance(content, list):
                    for item in content:
                        # 파일 타입 아이템 찾기
                        if isinstance(item, dict) and item.get('type') == 'file':
                            file_info = item.get('file', {})
                            if 'path' in file_info:
                                filepath_list.append(file_info['path'])
        
        # 현재 메시지의 파일들도 추가
        files = message.get("files", [])
        filepath_list.extend(files)
        
        print("Filepath list:", filepath_list)  # 디버깅용
        
        if filepath_list:
            # 모든 이미지 처리
            image_urls = []
            for file_path in filepath_list:
                try:
                    image_url = convert_to_url(file_path)
                    image_urls.append({"type": "image_url", "image_url": image_url})
                except Exception as e:
                    print(f"이미지 처리 중 오류 발생: {e}")
                    continue
            
            if not image_urls:
                return "이미지 처리 중 오류가 발생했습니다."
            
            # 메시지 구성
            content = [
                {"type": "text", "text": text if text else "이 이미지들에 대해 설명해주세요."},
                *image_urls
            ]
            
            try:
                # API 호출
                response = model.invoke([
                    HumanMessage(content=content)
                ])
                return response.content
            except Exception as e:
                return f"모델 응답 생성 중 오류가 발생했습니다: {str(e)}"
        
        return text if text else "이미지를 업로드해주세요."
    
    return "텍스트나 이미지를 입력해주세요."

# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=multimodal_bot,
    multimodal=True,
    title="멀티모달 챗봇",
    description="텍스트와 이미지를 함께 처리할 수 있는 챗봇입니다. 이전 대화의 이미지들도 함께 고려합니다.",
    analytics_enabled=False,  
    textbox=gr.MultimodalTextbox(placeholder="텍스트를 입력하거나 이미지를 업로드해주세요.", file_count="multiple", file_types=["image"]),
)

# 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


History: []
Filepath list: ['/private/var/folders/y2/5m8dl9kj7yl_xvfwnl0xlj8m0000gn/T/gradio/64acc6368053ea48efe74d877ce608aee21abedc26ea8842fddb22545d2b34f5/20230816 15h photoism_key square.jpeg', '/private/var/folders/y2/5m8dl9kj7yl_xvfwnl0xlj8m0000gn/T/gradio/8fc633a591dd278321a4ade8be3015f1d508529f23f2ea8a8a8981d949f4072d/avatar-profile.png']
History: [{'role': 'user', 'metadata': None, 'content': [{'file': {'path': '/private/var/folders/y2/5m8dl9kj7yl_xvfwnl0xlj8m0000gn/T/gradio/64acc6368053ea48efe74d877ce608aee21abedc26ea8842fddb22545d2b34f5/20230816 15h photoism_key square.jpeg', 'url': '/gradio_api/file=/private/var/folders/y2/5m8dl9kj7yl_xvfwnl0xlj8m0000gn/T/gradio/64acc6368053ea48efe74d877ce608aee21abedc26ea8842fddb22545d2b34f5/20230816 15h photoism_key square.jpeg', 'size': None, 'orig_name': None, 'mime_type': 'image/jpeg', 'is_stream': False, 'meta': {'_type': 'gradio.FileData'}}, 'alt_text': None, 'type': 'file'}, {'file': {'path': '/private/var/folders/y2/5m8dl9kj7yl_xvf

In [29]:
demo.close()

Closing server running on port: 7863


### 7) PDF 뷰어
- 설치: pip install gradio_pdf 또는 uv add gradio_pdf

In [30]:
from gradio_pdf import PDF

def answer_invoke(message, history):
    # TODO: PDF 처리 로직 추가 필요
    # 현재는 사용자 메시지만 반환하는 예시입니다
    # 실제 구현 시 PDF 파싱 라이브러리(PyPDF2, pdfplumber 등) 필요
    return message

with gr.Blocks(
    analytics_enabled=False,  
) as demo:
    with gr.Row():
        # API Key Section
        api_key_input = gr.Textbox(
            label="Enter OpenAI API Key",
            type="password",
            placeholder="sk-..."
        )
        
    with gr.Row():
        # PDF Upload and Chat Interface
        with gr.Column(scale=2):
            pdf_file = PDF(
                label="Upload PDF File",
                height=600,  # PDF 뷰어 높이 설정
            )
        with gr.Column(scale=1):
            chatbot = gr.ChatInterface(
                fn=answer_invoke,
                title="PDF-based Chatbot",
                description="Upload a PDF file and ask questions about its contents.",
            )


demo.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [ ]:
demo.close()

## Memory 추가

In [21]:
# chat_history 플레이스홀더를 사용
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 메시지 플레이스홀더가 있는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 파이썬(Python) 코드 작성을 도와주는 AI 어시스턴트입니다."),
    MessagesPlaceholder("chat_history"),
    ("system", "이전 대화 내용을 참고하여 질문에 대해서 친절하게 답변합니다."),
    ("human", "{user_input}")
])

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()


# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history):

    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    # history_messages.append(HumanMessage(content=message)) # 나의 질문 = 이 line 필요 없다고 하신 이유 이해 못함/앞뒤 설명 못 들음
    response = chain.invoke({
        "chat_history": history_messages,
        "user_input": message
    })
    return response
    

# Gradio ChatInterface 객체 생성
demo = gr.ChatInterface(
    fn=answer_invoke,         # 메시지 처리 함수
    title="파이썬 코드 어시스턴트", # 채팅 인터페이스의 제목
    )

# Gradio 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [1]:
# Gradio 인터페이스 종료
demo.close()

NameError: name 'demo' is not defined

---

# [실습 프로젝트]

### **다음과 같은 요구사항을 Gradio ChatInterface로 구현합니다**

- 주제: 맞춤형 여행 일정 계획 어시스턴트
- 기능: 
   - OpenAI Chat Completion API와 LangChain을 활용하여 사용자의 선호도에 맞는 여행 일정을 생성
   - LCEL을 사용하여 단계별 프롬프트 체인 구성 (사용자 입력 분석 -> 일정 생성 -> 세부 계획 수립)
   - 채팅 히스토리 사용하여 답변 생성
   - Gradio 인터페이스를 통해 사용자와 대화형으로 상호작용

- 주요 포인트:

   1. **모델 매개변수 최적화**
      - temperature=0.7: 적당한 창의성을 유지하면서 일관된 응답 생성
      - top_p=0.9: 높은 확률의 토큰만 선택하여 응답의 품질 향상
      - presence_penalty와 frequency_penalty: 반복적인 응답을 줄이고 다양한 제안 생성

   2. **시스템 프롬프트 설계**
      - 여행 플래너로서의 역할과 응답 가이드라인을 명확히 정의
      - 구체적인 정보를 포함하도록 지시
      - 한국어 응답 명시

   3. **메모리 관리**
      - Gradio 또는 LangChain 메모리 기능을 사용하여 대화 컨텍스트 유지
      - 이전 대화 내용을 바탕으로 연속성 있는 응답 생성

### 단계별 구현 가이드

- **Step 1**: 시스템 프롬프트 작성
   - 여행 플래너의 역할 명시
   - 구체적인 정보 포함 지시 (날짜, 장소, 예산 등)

- **Step 2**: 채팅 히스토리 관리
   - MessagesPlaceholder 사용
   - HumanMessage/AIMessage 변환

- **Step 3**: Gradio 인터페이스 설정
   - 제목과 설명 추가

**Step 4**: 테스트
   - "서울에서 2박 3일 여행 계획 짜줘" 등의 질문 시도

In [8]:
# 여기에 코드를 작성하세요.

# 환경 변수 로드
from dotenv import load_dotenv

load_dotenv()

# 필수 라이브러리 임포트
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import gradio as gr

# 메시지 플레이스홀더가 있는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 여행 일정을 계획해주는 전문 AI 어시스턴트입니다. 사용자의 선호도, 예산, 일정에 맞춰 구체적이고 실용적인 여행 계획을 제공합니다.",
        ),
        MessagesPlaceholder("chat_history"),
        ("human", "{user_input}"),
    ]
)


# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history, temperature):
    # Temperature에 따라 매번 새로운 모델 인스턴스 생성
    model = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=temperature,  # 사용자가 선택한 값 사용
        top_p=0.9,
        presence_penalty=0.3,
        frequency_penalty=0.3,
    )

    # 체인 생성
    chain = prompt | model | StrOutputParser()

    # Gradio history를 LangChain 메시지 형식으로 변환
    print(f"\n{'=' * 50}")
    print(f"📝 현재 입력 메시지: {message}")
    print(f"📚 History 개수: {len(history)}개")

    history_messages = []
    for msg in history:
        if msg["role"] == "user":
            # content가 리스트인 경우 텍스트만 추출
            content = msg["content"]
            if isinstance(content, list):
                text_content = " ".join(
                    [
                        item.get("text", "")
                        for item in content
                        if item.get("type") == "text"
                    ]
                )
                history_messages.append(HumanMessage(content=text_content))
            else:
                history_messages.append(HumanMessage(content=content))
        elif msg["role"] == "assistant":
            content = msg["content"]
            if isinstance(content, list):
                text_content = " ".join(
                    [
                        item.get("text", "")
                        for item in content
                        if item.get("type") == "text"
                    ]
                )
                history_messages.append(AIMessage(content=text_content))
            else:
                history_messages.append(AIMessage(content=content))

    # 변환된 히스토리 출력
    print(f"\n🔄 LangChain 형식으로 변환된 메시지:")
    for i, msg in enumerate(history_messages, 1):
        role = "👤 사용자" if isinstance(msg, HumanMessage) else "🤖 AI"
        content_preview = (
            msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
        )
        print(f"  {i}. {role}: {content_preview}")
    print(f"{'=' * 50}\n")

    # 체인 실행
    response = chain.invoke({"chat_history": history_messages, "user_input": message})
    return response


# 초기 환영 메시지 정의
initial_message = """안녕하세요! 🌍 맞춤형 여행 일정을 계획해드리는 AI 어시스턴트입니다.

완벽한 여행 계획을 위해 다음 정보를 알려주세요:

📍 **여행지**: 어디로 가고 싶으신가요?
📅 **여행 기간**: 며칠 동안 여행하실 계획인가요?
💰 **예산**: 대략적인 예산 범위는 어떻게 되시나요?
👥 **여행 인원**: 혼자, 가족, 친구들과 함께 가시나요?
🎯 **여행 스타일**: 휴양, 관광, 액티비티, 맛집 투어 등 선호하시는 스타일이 있나요?

위 정보를 자유롭게 말씀해주시면, 맞춤형 여행 일정을 상세히 계획해드리겠습니다! ✈️"""

# Gradio Blocks를 사용하여 커스텀 인터페이스 생성
with gr.Blocks(analytics_enabled=False) as demo:
    gr.Markdown("# 🌍 맞춤형 여행 일정 계획 어시스턴트")
    gr.Markdown(
        "여행지, 기간, 예산, 선호도를 알려주시면 맞춤형 여행 일정을 계획해드립니다!"
    )

    # Temperature 슬라이더 추가
    temperature_slider = gr.Slider(
        minimum=0.0,
        maximum=1.0,
        value=0.7,
        step=0.1,
        label="🎨 창의성 조절 (Temperature)",
        info="낮을수록(0.0) 일관되고 예측 가능한 답변, 높을수록(1.0) 창의적이고 다양한 답변을 생성합니다.",
        render=False,  # ChatInterface 내부에 표시
    )

    # 초기 메시지가 있는 Chatbot 생성 (최신 Gradio 형식)
    chatbot = gr.Chatbot(
        value=[{"role": "assistant", "content": initial_message}], height=500
    )

    # ChatInterface 생성
    gr.ChatInterface(
        fn=answer_invoke,
        chatbot=chatbot,  # 초기 메시지가 설정된 chatbot 사용
        additional_inputs=[temperature_slider],
        examples=[
            ["서울에서 2박 3일 여행 계획 짜줘", 0.7],
            ["제주도 가족 여행 일정 추천해줘", 0.3],
            ["유럽 배낭여행 1주일 계획 도와줘", 0.1],
        ],
    )

# Gradio 인터페이스 실행
demo.launch()


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [9]:
# Gradio 인터페이스 종료
demo.close()

Closing server running on port: 7866


[예시 답안]

In [ ]:
# 필수 라이브러리 임포트
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import gradio as gr

# 메시지 플레이스홀더가 있는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "여행 일정을 계획해주는 AI 어시스턴트입니다."),
    MessagesPlaceholder("chat_history"),
    ("system", "사용자의 선호도에 맞는 여행 일정을 생성합니다."),
    ("human", "{user_input}")
])

# LLM 모델 정의
model = ChatOpenAI(
    model="gpt-4.1-nano", 
    temperature=0.7, 
    top_p=0.9,
    presence_penalty=0.3,
    frequency_penalty=0.3,
    )

# 프롬프트 템플릿 + LLM 모델 + 출력파서를 연결하여 체인 생성
chain = prompt | model | StrOutputParser()

# 사용자 메시지를 처리하고 AI 응답을 생성하는 함수 (chat_history 사용)
def answer_invoke(message, history):

    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    history_messages.append(HumanMessage(content=message))
    response = chain.invoke({
        "chat_history": history_messages,
        "user_input": message
    })
    return response

# Gradio ChatInterface 객체 생성
demo = gr.ChatInterface(
    fn=answer_invoke,         # 메시지 처리 함수
    title="맞춤형 여행 일정 계획 어시스턴트", # 채팅 인터페이스의 제목
    )

# Gradio 인터페이스 실행
demo.launch()

In [ ]:
# Gradio 인터페이스 종료
demo.close()